In [3]:
import numpy as np

def calculate_probabilities(data, target, target_value):
    probabilities = []
    for col in range(data.shape[1]):
        unique_values, counts = np.unique(data[:, col], return_counts=True)
        for value in unique_values:
            subset = data[data[:, col] == value]
            target_subset = target[data[:, col] == value]
            class_count = np.sum(target_subset == target_value)
            prob = class_count / len(target_subset)
            probabilities.append((prob, col, value))
    return probabilities

def induce_rule(data, target, target_value):
    rule = []
    remaining_data = data.copy()
    remaining_target = target.copy()

    while True:
        probabilities = calculate_probabilities(remaining_data, remaining_target, target_value)
        max_prob, max_col, max_value = max(probabilities, key=lambda x: x[0])

        mask = remaining_data[:, max_col] == max_value
        remaining_data = remaining_data[mask]
        remaining_target = remaining_target[mask]

        rule.append((max_col, max_value))

        if np.all(remaining_target == target_value):
            break

    return rule

def prism(data, target, classes):
    rules = {i: [] for i in classes}
    data_remaining = data.copy()
    target_remaining = target.copy()

    for class_value in classes:
        while True:
            rule = induce_rule(data_remaining, target_remaining, class_value)
            rules[class_value].append(rule)

            indices_to_remove = np.array([True] * len(target_remaining))
            for col, value in rule:
                indices_to_remove = np.logical_and(indices_to_remove, data_remaining[:, col] == value)

            data_remaining = data_remaining[~indices_to_remove]
            target_remaining = target_remaining[~indices_to_remove]

            if np.sum(target_remaining == class_value) == 0:
                break

    return rules


In [4]:
# Exemplo de uso
data = np.array([[1, 2], [2, 1], [1, 1], [2, 2]])
target = np.array([0, 0, 1, 1])
classes = np.unique(target)

rules = prism(data, target, classes)
print(rules)

{0: [[(0, 1), (1, 2)], [(0, 2), (1, 1)]], 1: [[(0, 1)], [(0, 2)]]}
